In [15]:
from janome.analyzer import Analyzer
from janome.tokenfilter import *

import re
import neologdn
import emoji

import collections
import unicodedata
import string

In [16]:
def stop_word(text):

    stop_words = [ 'てる', 'いる', 'なる', 'れる', 'する', 'ある', 'こと', 'これ', 'さん', 'して', \
             'くれる', 'やる', 'くださる', 'そう', 'せる', 'した',  '思う',  \
             'それ', 'ここ', 'ちゃん', 'くん', 'て','に','を','は','の', 'が', 'と', 'た', 'し', 'で', \
             'ない', 'も', 'な', 'い', 'か', 'ので', 'よう', 'ん', 'みたい','の','私','自分','たくさん',\
             'ん','もの','こと']

    word_list = []
    for i in text:
        if i not in stop_words:
            word_list.append(i)
    return word_list

In [25]:
def wakati(text,word_list):
    a = Analyzer(token_filters=[CompoundNounFilter()])
    for token in a.analyze(text):
        if token.part_of_speech.split(',')[0] in ['名詞','形容詞','形容動詞']:
#     感情分析ではこっちを使用
#        if token.part_of_speech.split(',')[0] in ['名詞','形容詞','形容動詞','動詞']:
#     word2vecのモデル作成するときはこっちを使用
#        if token.part_of_speech.split(',')[0] in ['名詞']:
            word_list.append(token.base_form)
    return " ".join(word_list)

In [26]:
def cleansing(text):  
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'\n', '', text)
    text = neologdn.normalize(text)
    text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)
    text = unicodedata.normalize("NFKC", text) 
    table = str.maketrans("", "", string.punctuation  + "「」、。・")
    text = text.translate(table)
    return text

In [27]:
def main():
    text_file = 'ayane.txt'
    with open(text_file, encoding='utf-8') as f:
        s = f.read()
    s_after_cleansing = cleansing(s)
    word_list = []
    wakati(s_after_cleansing,word_list)
    s_after_wakati = " ".join(word_list)
    s_after_rmstop = stop_word(word_list)
#     ベクトル化用にテキストファイルで保存
    with open('名詞＿形容詞＿形容動詞.txt', 'w', encoding='utf-8') as f:
        f.write(s_after_wakati)

In [28]:
if __name__ == '__main__':
    main()